[catboost](#catboost)

In [1]:
import numpy as np
import pandas as pd

In [2]:
import pickle
x = open('C:/Users/Forge-15 1650/Desktop/190003/data analytic/final_train_bkp', 'rb')
final_train = pickle.load(x)

#train data
test = final_train[final_train['target'].isnull()] #seperating the test from the final dataset

#train data
train = final_train[final_train['target'].notnull()]
target_cols = ['target', 'ret', 'transactionRevenue_sum', 'fullVisitorId']
# ['target', 'ret', 'transactionRevenue_sum', 'fullVisitorId']
target_test = test['transactionRevenue_sum'].astype('float').apply(lambda x: np.log1p(x))

#target
target_cols = ['target', 'ret', 'transactionRevenue_sum', 'fullVisitorId']
target_test = test['transactionRevenue_sum'].astype('float').apply(lambda x: np.log1p(x))

# train data 
train_x = train.drop(target_cols, axis = 1)
train_x_id = train['fullVisitorId'].astype('str')
train_y = train['target']
train_ret = train['ret']
test_x = test.drop(target_cols, axis = 1)
test_x_id = test['fullVisitorId'].astype('str')
test_y = target_test
test_ret = test['ret']

In [3]:
#correct dtype train
train_x['isMobile'] = train_x['isMobile'].astype('object')
train_x['isTrueDirect'] = train_x['isTrueDirect'].astype('object')

#correct dtype test
test_x['isMobile'] = test_x['isMobile'].astype('object')
test_x['isTrueDirect'] = test_x['isTrueDirect'].astype('object')

#taking cat and numerical columns seperately
cat_cols = [x for x in train_x.columns if train_x[x].dtype == 'object']
num_cols = set(train_x.columns) - set(cat_cols)

print(num_cols)
print('\n')
print(cat_cols)

# fill missing in train
for col in cat_cols:
    train_x[col].fillna('missing', inplace=True) 
    test_x[col].fillna('missing', inplace=True)

for col in num_cols:       
    train_x[col].fillna(0, inplace=True)
    test_x[col].fillna(0, inplace=True)


{'sessionQualityDimMin', 'first_ses_from_the_period_start', 'bounces_mean', 'transactions', 'last_ses_from_the_period_end', 'newVisits', 'pageviews_min', 'timeOnSite_min', 'pageviews_max', 'hits_max', 'hits_sum', 'sessionQualityDimSum', 'sessionQualityDimMean', 'sessionQualityDimMax', 'pageviews_mean', 'visitNumber_max', 'hits_min', 'interval_dates', 'hits_mean', 'visitStartTime_counts', 'timeOnSite_max', 'pageviews_sum', 'timeOnSite_mean', 'timeOnSite_sum'}


['channelGrouping', 'browser', 'operatingSystem', 'isMobile', 'deviceCategory', 'continent', 'subContinent', 'country', 'region', 'metro', 'city', 'networkDomain', 'campaign', 'source', 'medium', 'keyword', 'googleAds', 'referralPath', 'isTrueDirect', 'customDimensions_value']


In [4]:
from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    print("transform column {}".format(col))
    lbe = LabelEncoder()
    lbe.fit(pd.concat([train_x[col],test_x[col]]).astype("str"))
    train_x[col] = lbe.transform(train_x[col].astype("str"))
    test_x[col] = lbe.transform(test_x[col].astype("str"))
    
#normalization
from sklearn.preprocessing import StandardScaler
for col in num_cols:
    norm = StandardScaler()
    norm.fit(train_x[col].values.reshape(-1,1))
    train_x[col] = norm.transform(train_x[col].values.reshape(-1,1))
    test_x[col] = norm.transform(test_x[col].values.reshape(-1,1))

transform column channelGrouping
transform column browser
transform column operatingSystem
transform column isMobile
transform column deviceCategory
transform column continent
transform column subContinent
transform column country
transform column region
transform column metro
transform column city
transform column networkDomain
transform column campaign
transform column source
transform column medium
transform column keyword
transform column googleAds
transform column referralPath
transform column isTrueDirect
transform column customDimensions_value


## catboost

training data  included all features except target columns below: 
- ['target', 'ret', 'transactionRevenue_sum', 'fullVisitorId']

In [5]:
train_x[['channelGrouping', 'first_ses_from_the_period_start', 'last_ses_from_the_period_end', 'interval_dates',
         'visitStartTime_counts', 'visitNumber_max', 'isMobile', 'operatingSystem', 'deviceCategory', 'continent',
         'country', 'region', 'city', 'networkDomain', 'bounces_mean', 'hits_sum', 'hits_min', 'hits_max', 'hits_mean', 
         'pageviews_sum', 'pageviews_min', 'pageviews_max', 'pageviews_mean', 'sessionQualityDimMax', 'sessionQualityDimMean',
         'timeOnSite_max', 'timeOnSite_mean', 'timeOnSite_sum', 'transactions', 'campaign', 'source', 'medium', 'referralPath']]

,channelGrouping,first_ses_from_the_period_start,last_ses_from_the_period_end,interval_dates,visitStartTime_counts,visitNumber_max,isMobile,operatingSystem,deviceCategory,continent,...,sessionQualityDimMax,sessionQualityDimMean,timeOnSite_max,timeOnSite_mean,timeOnSite_sum,transactions,campaign,source,medium,referralPath
0,4,-0.018261,0.219949,-0.185269,-0.207383,-0.165488,0,6,0,4,...,-0.227094,-0.247510,0.163520,0.350777,0.058843,-0.092882,41,110,4,3571
1,4,0.875995,-0.677161,-0.185269,-0.207383,-0.165488,0,21,0,1,...,-0.227094,-0.247510,0.439200,0.736632,0.224314,-0.092882,41,0,3,3571
2,4,-1.326927,1.532793,-0.185269,-0.207383,-0.165488,1,0,1,1,...,-0.227094,-0.247510,0.184932,0.380746,0.071695,-0.092882,41,110,4,3571
3,4,-0.519916,0.723206,-0.185269,-0.207383,-0.165488,0,21,0,1,...,-0.227094,-0.247510,-0.345016,-0.360993,-0.246395,-0.092882,41,0,3,3571
4,7,-1.610472,1.817242,-0.185269,-0.207383,-0.165488,0,6,0,2,...,-0.227094,-0.247510,-0.355722,-0.375978,-0.252821,-0.092882,41,380,5,2900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1344562,2,0.090795,-0.633399,-0.185269,-0.207383,-0.165488,1,0,1,1,...,0.135457,0.271037,0.104637,0.268362,0.023499,-0.092882,41,0,3,3571
1344563,6,1.115917,-1.661794,-0.185269,-0.207383,-0.165488,0,6,0,1,...,8.353278,12.024767,1.424155,2.115218,0.815512,6.367849,41,0,3,3571
1344564,4,-1.370550,0.832609,-0.185269,-0.207383,-0.165488,0,5,0,3,...,-0.106244,-0.074661,-0.355722,-0.375978,-0.252821,-0.092882,41,110,4,3571
1344565,4,-0.236372,-0.305189,-0.185269,-0.207383,-0.165488,0,21,0,1,...,0.014606,0.098188,0.013636,0.140992,-0.031122,-0.092882,41,110,4,3571


## Parameters tuning


1. learning rate: controls how fast a model learns <br>
 - smaller value, higher iterations needed to train Catboost <br>
 - initial learning rates for grid search: [0.03, 0.05, 0.1]; iterations = 1000


2. Tree depth:  <br>
 - larger tree depth, more splits, captures more data information
 
 
3. L2 regularization: ridge regression is chosen
 - prevent overfitting

In [ ]:
grid_search_result

- best parameters from grid search was found to be:

In [ ]:
train_x.columns


### best catboost parameters found from running grid search are fitted to the model

In [7]:
from catboost import CatBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def rmse(y_true, y_pred):
    return round(np.sqrt(mean_squared_error(y_true, y_pred)), 5)


X_train, X_validation, y_train, y_validation = train_test_split(train_x, 
                                                                train_y, 
                                                                test_size=0.15,     #  train size 85%, test size 15%
                                                                random_state=1)     # fixed train-test splits to be deterministic
# use best parameters from Grid Search results
clf = CatBoostRegressor(iterations = 1000,                      
                        learning_rate = 0.03,             
                        depth = 8,
                        l2_leaf_reg = 1,
                        eval_metric='RMSE',               # square root of mean-squared error [accuracy test]
                        od_wait = 10)

clf.fit(X_train, y_train,
        eval_set = (X_validation, y_validation),
        use_best_model = True,
        verbose=True)

y_pred_train = clf.predict(X_train)
y_pred_validation = clf.predict(X_validation)
y_pred_test = clf.predict(test_x)

print(f"CatB: RMSE val: {rmse(y_validation, y_pred_validation)}  - RMSE train: {rmse(y_train, y_pred_train)}" , "= {}".format((rmse(y_validation, y_pred_validation)-(rmse(y_train, y_pred_train)))))

x = pd.DataFrame(data ={'fullVisitorId': test['fullVisitorId'].astype('str'), 'PredictedLogRevenue': y_pred_test})
x.to_csv('D:/datasets/results/submission_0521_catboost_WithValidationSet.csv', index = False)

0:	learn: 0.3127098	test: 0.2966407	best: 0.2966407 (0)	total: 387ms	remaining: 6m 26s
1:	learn: 0.3123044	test: 0.2965706	best: 0.2965706 (1)	total: 669ms	remaining: 5m 33s
2:	learn: 0.3118691	test: 0.2964545	best: 0.2964545 (2)	total: 836ms	remaining: 4m 37s
3:	learn: 0.3114848	test: 0.2964320	best: 0.2964320 (3)	total: 1.1s	remaining: 4m 33s
4:	learn: 0.3111170	test: 0.2962598	best: 0.2962598 (4)	total: 1.35s	remaining: 4m 29s
5:	learn: 0.3107165	test: 0.2962451	best: 0.2962451 (5)	total: 1.53s	remaining: 4m 13s
6:	learn: 0.3103071	test: 0.2961914	best: 0.2961914 (6)	total: 1.71s	remaining: 4m 2s
7:	learn: 0.3099001	test: 0.2961453	best: 0.2961453 (7)	total: 2.01s	remaining: 4m 9s
8:	learn: 0.3095553	test: 0.2960599	best: 0.2960599 (8)	total: 2.26s	remaining: 4m 8s
9:	learn: 0.3091827	test: 0.2960120	best: 0.2960120 (9)	total: 2.53s	remaining: 4m 10s
10:	learn: 0.3088333	test: 0.2960079	best: 0.2960079 (10)	total: 2.83s	remaining: 4m 14s
11:	learn: 0.3084764	test: 0.2959789	best: 0.

## from results above, best RMSE is 0.290167 at 147th iteration

- model is hence shrinked to first 148 iterations
- difference in RMSE value between validation and training sets is 0.00043.